In [12]:
import os 

path_images = './object_detection'
file_names = os.listdir(path_images)

xml_files = [i for i in file_names if i.endswith('.xml')]
jpg_files = [i for i in file_names if i.endswith('.jpg')]
txt_files = [i for i in file_names if i.endswith('.txt')]

print(len(xml_files), len(jpg_files), len(txt_files))


400 400 0


In [3]:
import os
import xml.etree.ElementTree as ET
import cv2

def convert_to_yolo(xml_file, image_width, image_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_lines = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Calcular coordenadas relativas yolo
        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        yolo_line = f"{class_name} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        yolo_lines.append(yolo_line)

    return yolo_lines

# Directorio donde se encuentran los archivos XML y las imágenes correspondientes
xml_dir = './object_detection'
image_dir = './object_detection'

# Iterar sobre los archivos XML en el directorio
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        # Obtener el nombre del archivo de la imagen correspondiente
        image_name = os.path.splitext(xml_file)[0] + '.jpg'
        image_path = os.path.join(image_dir, image_name)
        
        # Obtener el ancho y alto de la imagen
        image = cv2.imread(image_path)
        image_height, image_width, _ = image.shape

        # Convertir el archivo XML al formato YOLO
        yolo_lines = convert_to_yolo(os.path.join(xml_dir, xml_file), image_width, image_height)

        # Guardar las líneas en un archivo YOLO
        yolo_file = os.path.splitext(xml_file)[0] + '.txt'
        with open(os.path.join(xml_dir, yolo_file), 'w') as f:
            for line in yolo_lines:
                f.write(line + '\n')
